JAADOpenPose dataset stats:
=========================================================

In [1]:
# seeding

import pytorch_lightning as pl
import numpy as np

seed = 22742

pl.seed_everything(seed, workers=True)

Global seed set to 22742


chamfered-column


In [2]:
# DataModule to use

from pedestrians_video_2_carla.data.openpose.skeleton import BODY_25_SKELETON
from pedestrians_video_2_carla.data.base.base_transforms import BaseTransforms
from pedestrians_video_2_carla.data.openpose.jaad_openpose_datamodule import JAADOpenPoseDataModule

dm = JAADOpenPoseDataModule(
    batch_size=256,
    clip_offset=1,
    clip_length=1,
    transforms=BaseTransforms.hips_neck_bbox,
    data_nodes=BODY_25_SKELETON,
    input_nodes=BODY_25_SKELETON,
    strong_points=0,
    num_workers=0
)
dm.prepare_data()
dm.setup()

Subsets dir: /outputs/JAADOpenPoseDataModule/subsets/b770d34d6f1911954eea04ec81cf3999


Good practice: only get relevant stats on training data.

In [6]:
# we're only interested in raw projection_2d data, ignoring confidence
pose_2d = dm.train_set.projection_2d[..., :2]

In [12]:
# how many frames do we have?
total_frames = pose_2d.shape[0]
total_frames

36140

In [13]:
# how many frames where all joints are present?
all_joints = np.all(np.any(pose_2d, axis=-1), axis=-1).sum()
all_joints

6766

In [14]:
# how many frames where at least one joint is present?
at_least_one_joint = np.any(np.any(pose_2d, axis=-1), axis=-1).sum()
at_least_one_joint

28012

In [24]:
# how many frames where a particular joint is present?
separate_joints = np.sum(np.any(pose_2d, axis=-1), axis=(0,1))

tuple(zip(BODY_25_SKELETON._member_names_, separate_joints))

(('Nose', 22972),
 ('Neck', 27809),
 ('RShoulder', 27618),
 ('RElbow', 26791),
 ('RWrist', 24757),
 ('LShoulder', 27614),
 ('LElbow', 26829),
 ('LWrist', 24927),
 ('MidHip', 27848),
 ('RHip', 27789),
 ('RKnee', 27396),
 ('RAnkle', 26716),
 ('LHip', 27789),
 ('LKnee', 27363),
 ('LAnkle', 26692),
 ('REye', 20091),
 ('LEye', 19330),
 ('REar', 21977),
 ('LEar', 21767),
 ('LBigToe', 25117),
 ('LSmallToe', 24663),
 ('LHeel', 26300),
 ('RBigToe', 25581),
 ('RSmallToe', 25318),
 ('RHeel', 26450))

In [20]:
# what is percentage of frames where all joints are present?
all_joints_percentage = all_joints / total_frames
all_joints_percentage

0.1872163807415606

In [21]:
# what is percentage of frames where at least one joint is present?
at_least_one_joint_percentage = at_least_one_joint / total_frames
at_least_one_joint_percentage

0.7750968456004427

In [25]:
# what is percentage of frames where a particular joint is present?
separate_joints_percentage = separate_joints / total_frames
tuple(zip(BODY_25_SKELETON._member_names_, separate_joints_percentage))

(('Nose', 0.6356391809629219),
 ('Neck', 0.7694798007747649),
 ('RShoulder', 0.7641947980077477),
 ('RElbow', 0.74131156613171),
 ('RWrist', 0.6850304371887106),
 ('LShoulder', 0.7640841173215274),
 ('LElbow', 0.7423630326508024),
 ('LWrist', 0.6897343663530714),
 ('MidHip', 0.7705589374654123),
 ('RHip', 0.7689263973436635),
 ('RKnee', 0.7580520199225235),
 ('RAnkle', 0.7392363032650803),
 ('LHip', 0.7689263973436635),
 ('LKnee', 0.7571389042612064),
 ('LAnkle', 0.7385722191477587),
 ('REye', 0.5559214167127836),
 ('LEye', 0.5348644161593802),
 ('REar', 0.6081073602656336),
 ('LEar', 0.6022966242390703),
 ('LBigToe', 0.6949916989485335),
 ('LSmallToe', 0.6824294410625346),
 ('LHeel', 0.7277255118981738),
 ('RBigToe', 0.707830658550083),
 ('RSmallToe', 0.7005534034311013),
 ('RHeel', 0.7318760376314333))

In [31]:
# per-joint missing joint probability (rounded to 2 decimal places)

mjp = np.round(1 - separate_joints_percentage, 2)
tuple(mjp)

(0.36,
 0.23,
 0.24,
 0.26,
 0.31,
 0.24,
 0.26,
 0.31,
 0.23,
 0.23,
 0.24,
 0.26,
 0.23,
 0.24,
 0.26,
 0.44,
 0.47,
 0.39,
 0.4,
 0.31,
 0.32,
 0.27,
 0.29,
 0.3,
 0.27)

In [32]:
# global missing joint probability
np.round(1  - (separate_joints.sum() / (total_frames * len(BODY_25_SKELETON))), 2)

0.29

In [34]:
np.round((1 - separate_joints_percentage).mean(), 2)

0.29